#Zadanie 1a (3 pkt)
Celem zadania jest rozwiązanie problemu plecakowego dwoma metodami - brute force oraz według zadanej heurystyki. Należy zaimplementować metody klasy *KnapSack* - *solve_knapsack_brute_force* oraz *solve_knapsack_pw_ratio*. Poprzez rozwiązanie problemu rozumiemy podanie które przedmioty (indeksy w tablicy) należy spakować do plecaka oraz jaka jest sumaryczna wartość i masa plecaka. Punktacja wygląda następująco:


*   Rozwiązanie problemu metodą brute force. *Podpowiedź: do wygenerowania wszystkich permutacji można użyć funkcji product z biblioteki itertools* - **1.5 pkt**
*   Rozwiązanie problemu według heurystyki - do plecaka pakujemy przedmioty według stosunku wartości do wagi - **1 pkt**
*   Dla metody brute force proszę wygenerować wykres zależności czasu wykonywania metody od liczby elementów w tablicach *weights* i *profits* (do obu tablic należy stopniowo dopisywać po jednym elemencie, np. 10-krotnie, wartości elementów nie mają znaczenia). Proszę również odpowiedzieć na pytania (w osobnej komórce tekstowej) - czy obie metody mają takie same rozwiązania? Jakie są Pani / Pana wnioski? - **0.5 pkt**




In [1]:
import numpy as np
import itertools
from matplotlib import pyplot as plt
import gc
import time

In [2]:
weights = np.array([8, 3, 5, 2])
capacity = 9
profits = np.array([16, 8, 9, 6])

In [4]:
class KnapSack:
  def __init__(self, profits, weights, capacity):
    self.profits = profits
    self.weights = weights
    self.capacity = capacity

  def solve_knapsack_brute_force(self):   # -> best_combination, max_profit, summed_weights
    item_count = len(self.weights)

    if item_count == 0 or capacity == 0:
      return [], 0, 0

    max_profit = 0;
    best_combination = [];
  
    all_combinations = itertools.product([0, 1], repeat=item_count)
    #  all combinations of binary numbers of len=item_count
    #  ==> [[0, 0, 0, .., 0], [1, 0, 0, .., 0], ..]

    for combination in all_combinations:
      comb_weight_sum = 0
      comb_profit_sum = 0

      for i in range(item_count):
        if combination[i] == 1:       # if item is being taken
          comb_weight_sum += self.weights[i]
          comb_profit_sum += self.profits[i]  # we add their weight and profits to the sum
      
      if comb_weight_sum <= capacity and comb_profit_sum > max_profit:
        # if weight of combination items does not exceed capacity and profit is better than earlier one
        max_profit = comb_profit_sum
        best_combination = combination
    
    return best_combination, max_profit, sum(self.weights[i] for i in range(item_count) if best_combination[i] == 1)


  def solve_knapsack_pw_ratio(self):
    pass

In [ ]:
class TimeMeasurer:

    @staticmethod
    def manage_gc(disable):
        if disable:
            gc.disable();

    @staticmethod
    def measure(func, *args, disable_gc=False, **kwargs):
        TimeMeasurer.manage_gc(disable_gc)

        start_time = time.time()
        result = func(args, kwargs)
        end_time = time.time()

        TimeMeasurer.manage_gc(disable_gc)

        return end_time - start_time